In [25]:
import math

def twd97_to_lonlat(x=174458.0,y=2525824.0):
    """
    Parameters
    ----------
    x : float
        TWD97 coord system. The default is 174458.0.
    y : float
        TWD97 coord system. The default is 2525824.0.
    Returns
    -------
    list
        [longitude, latitude]
    """
    
    a = 6378137
    b = 6356752.314245
    long_0 = 121 * math.pi / 180.0
    k0 = 0.9999
    dx = 250000
    dy = 0
    
    e = math.pow((1-math.pow(b, 2)/math.pow(a,2)), 0.5)
    
    x -= dx
    y -= dy
    
    M = y / k0
    
    mu = M / ( a*(1-math.pow(e, 2)/4 - 3*math.pow(e,4)/64 - 5 * math.pow(e, 6)/256))
    e1 = (1.0 - pow((1   - pow(e, 2)), 0.5)) / (1.0 +math.pow((1.0 -math.pow(e,2)), 0.5))
    
    j1 = 3*e1/2-27*math.pow(e1,3)/32
    j2 = 21 * math.pow(e1,2)/16 - 55 * math.pow(e1, 4)/32
    j3 = 151 * math.pow(e1, 3)/96
    j4 = 1097 * math.pow(e1, 4)/512
    
    fp = mu + j1 * math.sin(2*mu) + j2 * math.sin(4* mu) + j3 * math.sin(6*mu) + j4 * math.sin(8* mu)
    
    e2 = math.pow((e*a/b),2)
    c1 = math.pow(e2*math.cos(fp),2)
    t1 = math.pow(math.tan(fp),2)
    r1 = a * (1-math.pow(e,2)) / math.pow( (1-math.pow(e,2)* math.pow(math.sin(fp),2)), (3/2))
    n1 = a / math.pow((1-math.pow(e,2)*math.pow(math.sin(fp),2)),0.5)
    d = x / (n1*k0)
    
    q1 = n1* math.tan(fp) / r1
    q2 = math.pow(d,2)/2
    q3 = ( 5 + 3 * t1 + 10 * c1 - 4 * math.pow(c1,2) - 9 * e2 ) * math.pow(d,4)/24
    q4 = (61 + 90 * t1 + 298 * c1 + 45 * math.pow(t1,2) - 3 * math.pow(c1,2) - 252 * e2) * math.pow(d,6)/720
    lat = fp - q1 * (q2 - q3 + q4)
    
    
    q5 = d
    q6 = (1+2*t1+c1) * math.pow(d,3) / 6
    q7 = (5 - 2 * c1 + 28 * t1 - 3 * math.pow(c1,2) + 8 * e2 + 24 * math.pow(t1,2)) * math.pow(d,5) / 120
    lon = long_0 + (q5 - q6 + q7) / math.cos(fp)
    
    lat = (lat*180) / math.pi
    lon = (lon*180) / math.pi
    return [lat, lon]

In [26]:
import csv
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

from bs4 import BeautifulSoup
from datetime import datetime

In [33]:
f = open('filter_disaster_20231115', 'r')
reader = csv.reader(f)

# 將lists轉換成DataFrame
df = pd.DataFrame(reader)

path = r'chromedriver.exe'
service=Service(executable_path=path)

chrome=webdriver.Chrome(service=service)
chrome.get('https://www.google.com/maps/')

time.sleep(5)

# 按下圖層轉換
input_xpath_2 = '/html/body/div[1]/div[3]/div[8]/div[23]/div[5]/div/div[2]/button'
input_button_2 = chrome.find_element(By.XPATH, input_xpath_2)
input_button_2.click()
time.sleep(2)

for i in range(1,len(df[10])):
    le = twd97_to_lonlat(x=float(df[10][i]),y=float(df[11][i]))
    final = str(le[0]) +','+str(le[1])

    # 輸入座標
    un_xpath = '/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input'
    element = chrome.find_element(By.XPATH, un_xpath)
    element.send_keys(final)
    time.sleep(2)   
    # 按下搜尋
    input_xpath = '/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/div[1]/button'
    input_button = chrome.find_element(By.XPATH, input_xpath)
    input_button.click()
    time.sleep(2)

    # 按下分享
    input_xpath_3 = '/html/body/div[1]/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[4]/div[5]/button'
    input_button_3 = chrome.find_element(By.XPATH, input_xpath_3)
    input_button_3.click()
    time.sleep(2)

    # 按下嵌入地圖
    input_xpath_4 = '/html/body/div[1]/div[3]/div[1]/div/div[2]/div/div[3]/div/div/div/div[2]/button[2]'
    input_button_4 = chrome.find_element(By.XPATH, input_xpath_4)
    input_button_4.click()

    # 取得災難點html
    p = chrome.page_source
    soup = BeautifulSoup(p, 'lxml')
    p_authors = soup.find('input',{'class':'yA7sBe'})

    date=datetime.now().strftime('%Y%m%d')

    with open('googlemap_figure/'+date+'-'+df[4][i]+'.html','w') as file:
        file.write(p_authors['value'])


    # 按下關閉嵌入地圖
    input_xpath_5 = '/html/body/div[1]/div[3]/div[1]/div/div[2]/div/div[2]/button'
    input_button_5 = chrome.find_element(By.XPATH, input_xpath_5)
    input_button_5.click()

    # 清除搜尋框
    un_xpath = '/html/body/div[1]/div[3]/div[8]/div[3]/div[1]/div[1]/div/div[2]/form/input'
    element = chrome.find_element(By.XPATH, un_xpath)
    element.clear()